In [1]:
import argparse
import logging
import os
import sys

import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
from torch import optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, random_split

sys.path.append("..")
sys.path.append("../scripts/")
import superlayer.utils

from scripts import eval_net, train_net, get_args

from superlayer.models import SuperNet, AESuperNet
from superlayer.utils import BrainD, dice_coeff, one_hot, plot_img_array, plot_side_by_side

In [2]:
dir_img = '/home/gid-dalcaav/projects/neuron/data/t1_mix/proc/resize256-crop_x32-slice100/train/vols/'
dir_mask = '/home/gid-dalcaav/projects/neuron/data/t1_mix/proc/resize256-crop_x32-slice100/train/asegs/'

dir_train = '/home/vib9/src/SL-Net/jupyter/partitions/train.txt'
dir_val = '/home/vib9/src/SL-Net/jupyter/partitions/val.txt'

dir_checkpoint_1 = 'checkpoints_1/'
dir_checkpoint_2 = 'checkpoints_2/'

In [3]:
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
args = get_args()
device = torch.device('cuda')
logging.info(f'Using device {device}')

INFO: Using device cuda


In [4]:
target_label_numbers = [0,2,3,4,10,16,17,28,31,41,42,43,49,53,63]
val_percent = 0.1
batch_size = args.batchsize
lr = args.lr
img_scale = args.scale
writer = SummaryWriter(comment=f'LR_{lr}_BS_{batch_size}_SCALE_{img_scale}')

In [ ]:
net1 = SuperNet(input_ch=1, out_ch=15, use_bn=True, superblock_size=256, depth=4)

logging.info(f'Network:\n'
             f'\t{1} input channels\n'
             f'\t{15} output channels (classes)\n')

net1.to(device=device)
train_scores1, val_scores1, train_var_1, val_var_1 = train_net(net=net1,
                                                                epochs=5,
                                                                batch_size=args.batchsize,
                                                                lr=args.lr,
                                                                device=device,
                                                                img_scale=args.scale,
                                                                val_percent=args.val / 100,
                                                                checkpoint=2,
                                                                target_label_numbers=target_label_numbers,
                                                                writer=writer,
                                                                train_path=dir_train,
                                                                val_path=dir_val)

INFO: Network:
	1 input channels
	15 output channels (classes)

INFO: Creating dataset with 7329 examples
INFO: Creating dataset with 7329 examples
INFO: Starting training:
        Epochs:          5
        Batch size:      8
        Learning rate:   0.001
        Training size:   7329
        Validation size: 7329
        Checkpoints:     True
        Device:          cuda
        Images scaling:  1
    
Validation round:  43%|████▎     | 398/916 [00:16<00:23, 21.64batch/s]


Validation round:  91%|█████████ | 835/916 [00:32<00:03, 22.93batch/s]


Validation round: 100%|█████████▉| 915/916 [00:35<00:00, 26.33batch/s]
                                                                      INFO: Validation Dice Loss: 0.32837501151983395
Validation round:  34%|███▍      | 313/916 [00:12<00:21, 28.17batch/s]


Validation round:  79%|███████▉  | 722/916 [00:28<00:08, 23.95batch/s]


Validation round: 100%|█████████▉| 914/916 [00:36<00:00, 23.41batch/s]
                                                                      INFO: Validation Dice Loss: 0.34783988049066744
Validation round:  22%|██▏       | 206/916 [00:08<00:25, 27.73batch/s]


...

Validation round:  76%|███████▌  | 695/916 [00:30<00:13, 16.88batch/s]


Validation round: 100%|█████████▉| 913/916 [00:39<00:00, 27.68batch/s]
                                                                      INFO: Validation Dice Loss: 0.34175512767254523
Validation round:  19%|█▉        | 176/916 [00:07<00:28, 26.27batch/s]


Validation round:  67%|██████▋   | 616/916 [00:25<00:10, 27.38batch/s]


Validation round: 100%|██████████| 916/916 [00:37<00:00, 25.02batch/s]
                                                                      INFO: Validation Dice Loss: 0.3423132739928612
Validation round:  12%|█▏        | 113/916 [00:05<00:33, 24.08batch/s]


Validation round:  51%|█████     | 469/916 [00:19<00:16, 26.31batch/s]

In [ ]:
net2 = AESuperNet(input_ch=1, out_ch=15, use_bn=True, superblock_size=256, depth=4)

logging.info(f'Network:\n'
             f'\t{1} input channels\n'
             f'\t{15} output channels (classes)\n')

net2.to(device=device)
train_scores2, val_scores2, train_var_2, val_var_2 = train_net(net=net2,
                                                                epochs=5,
                                                                batch_size=args.batchsize,
                                                                lr=args.lr,
                                                                device=device,
                                                                img_scale=args.scale,
                                                                val_percent=args.val / 100,
                                                                checkpoint=2,
                                                                target_label_numbers=target_label_numbers,
                                                                writer=writer,
                                                                train_path=dir_train,
                                                                val_path=dir_val)